In [1]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

In [2]:
# Mesma separacao do exercicio anterior (SVM) salvas em csv
X_train = pd.read_csv('../data/X_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_train = pd.read_csv('../data/y_train.csv')
y_test = pd.read_csv('../data/y_test.csv')

In [4]:
# normalizando os dados

# Flagando paraa separar depois
X_train['tipo'] = 'train'
X_test['tipo'] = 'test'

#juntando datasets para normalizar em conjunto
X = pd.concat([X_train,X_test])

# Normalizando 0 e 1
types = X['tipo']
X = X.drop(['tipo'],axis=1)
X = (X - X.min()) / (X.max() - X.min())

# voltando a coluna de tipo para separa-los novamente
X['tipo'] = types
# voltando para o dado original
X_train = X.loc[X['tipo'] == 'train'].drop(['tipo'],axis=1)
X_test =  X.loc[X['tipo'] == 'test'].drop(['tipo'],axis=1)

In [11]:
def train_pred_classifier(X_train, X_test, y_train, y_test,
                          hl_size, activation, alpha,epocas):
    
    print(alpha)
    print(hl_size)
    # prepara classificador
    clf = MLPClassifier(solver='lbfgs', # resolve com hasseane, mais rapido
                        alpha=alpha,
                        hidden_layer_sizes=hl_size,
                        activation = activation,
                        max_iter = epocas,
                        random_state=1)
    # treina
    clf.fit(X_train,y_train)
    
    # predict
    y_pred = clf.predict(X_test)
    
    # calcula erros
    p = precision_score(y_test, y_pred, average='micro')
    r = recall_score(y_test, y_pred, average='micro')
    auc = roc_auc_score(y_test, y_pred, average='micro')
    return p,r,auc
    

In [12]:
tipo = 'Todas as Caracteristicas'
camadas_ocultas = [
  (10,1), #  uma camada oculta com 1000 neuronios
  (10, 15), # duas camadas ocultas com 1000 neuronios e 200
  (20, 15), # tres camadas ocultas
  (10, 30, 10) # tres camadas ocultas
]
funcao_ativacao = ['logistic', 'relu']
alpha = [0.001, 0.01, 0.1, 0.2, 0.3]
epocas = [500, 1000, 2000, 5000]


In [25]:
all_dicts = []
for t in tipo:
    for layer_conf in camadas_ocultas:
        for f in funcao_ativacao:
            for a in alpha:
                for epoch in epocas:
                    p,r,auc = train_pred_classifier(X_train.head(10), X_test.head(10),
                                          y_train.head(10), y_test.head(10),
                                          layer_conf, f, a,epoch)
                    one_line_response = {
                        'tipo' : tipo,
                        'camadas_ocultas' : layer_conf,
                        'funcao_ativacao' : f,
                        'num_epocas' : epoch,
                        'alpha': a,
                        'precision': p,
                        'recall': r,
                        'auc': auc
                    }
                    all_dicts.append(one_line_response)
                    break

0.001
(10, 1)


/home/favoretti/.local/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.01
(10, 1)
0.1
(10, 1)


KeyboardInterrupt: 

In [26]:
all_dicts

[{'alpha': 0.001,
  'auc': 0.20000000000000001,
  'camadas_ocultas': (10, 1),
  'funcao_ativacao': 'logistic',
  'num_epocas': 500,
  'precision': 0.20000000000000001,
  'recall': 0.20000000000000001,
  'tipo': 'Todas as Caracteristicas'},
 {'alpha': 0.01,
  'auc': 0.29999999999999999,
  'camadas_ocultas': (10, 1),
  'funcao_ativacao': 'logistic',
  'num_epocas': 500,
  'precision': 0.29999999999999999,
  'recall': 0.29999999999999999,
  'tipo': 'Todas as Caracteristicas'}]

In [36]:
pd.DataFrame(all_dicts)

,alpha,auc,camadas_ocultas,funcao_ativacao,num_epocas,precision,recall,tipo
0,0.001,0.2,"(10, 1)",logistic,500,0.2,0.2,Todas as Caracteristicas
1,0.010,0.3,"(10, 1)",logistic,500,0.3,0.3,Todas as Caracteristicas
